 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8890/notebooks/Transform%20Street%20Names.ipynb#Approach-to-transforming-and-auditing-street-names" data-toc-modified-id="Approach-to-transforming-and-auditing-street-names-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Approach to transforming and auditing street names</a></span></li><li><span><a href="http://localhost:8890/notebooks/Transform%20Street%20Names.ipynb#Define-structure-for-street-names" data-toc-modified-id="Define-structure-for-street-names-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define structure for street names</a></span></li><li><span><a href="http://localhost:8890/notebooks/Transform%20Street%20Names.ipynb#Transform-street-names" data-toc-modified-id="Transform-street-names-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Transform street names</a></span></li><li><span><a href="http://localhost:8890/notebooks/Transform%20Street%20Names.ipynb#Validate-transformed-street-names-against-the-defined-structure" data-toc-modified-id="Validate-transformed-street-names-against-the-defined-structure-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Validate transformed street names against the defined structure</a></span></li></ul></div>

## Approach to transforming and auditing street names

1. Establish assumptions about the data. We might assume that all the roads end with a particular suffix, or have a certain convention.
1. Transform the street names.
1. Validate the transformed street names against the assumptions in step 1.
1. Recursively follow steps 1 to 3 to get everything audited.

## Define structure for street names

## Transform street names

## Validate transformed street names against the defined structure